In [1]:
import os
import zipfile
import pandas as pd
import gc
import glob
from sqlalchemy import create_engine
from datetime import timedelta
from sklearn.model_selection import train_test_split

##### 在训练机器学习模型时，为了有尽可能多的数据集，所以单独选择故障磁盘数据集：故障发生7天前的数据都进行提取。

In [2]:
# 读取错误数据集
smartlog_data_fault_A_df = pd.read_csv('smartlog_data_fault_A_df.csv')
smartlog_data_fault_B_df = pd.read_csv('smartlog_data_fault_B_df.csv')
smartlog_data_fault_A_df = smartlog_data_fault_A_df[smartlog_data_fault_A_df['dt'] <= smartlog_data_fault_A_df['fault_time']]
smartlog_data_fault_B_df = smartlog_data_fault_B_df[smartlog_data_fault_B_df['dt'] <= smartlog_data_fault_B_df['fault_time']]
smartlog_data_fault_A_df = smartlog_data_fault_A_df.drop_duplicates()
smartlog_data_fault_B_df = smartlog_data_fault_B_df.drop_duplicates()
smartlog_data_fault_A_df

,serial_number,model,smart_5raw,smart_4raw,smart_192raw,smart_12raw,smart_5_normalized,smart_7_normalized,dt,fault_time,first_observation_time,time_diff
0,disk_69492,1,0.0,5.0,4.0,5.0,100.0,90.0,2017-07-22,2017-10-27,2017-07-09,110
1,disk_69492,1,0.0,5.0,4.0,5.0,100.0,90.0,2017-07-19,2017-10-27,2017-07-09,110
2,disk_69492,1,0.0,5.0,4.0,5.0,100.0,90.0,2017-07-29,2017-10-27,2017-07-09,110
3,disk_69492,1,0.0,5.0,4.0,5.0,100.0,90.0,2017-07-16,2017-10-27,2017-07-09,110
4,disk_69492,1,0.0,5.0,4.0,5.0,100.0,90.0,2017-07-20,2017-10-27,2017-07-09,110
...,...,...,...,...,...,...,...,...,...,...,...,...
334279,disk_73603,1,427.0,27.0,25.0,26.0,97.0,84.0,2018-12-01,2018-12-04,2018-12-01,3
334280,disk_73603,1,429.0,27.0,25.0,26.0,97.0,84.0,2018-12-04,2018-12-04,2018-12-01,3
334281,disk_73603,1,428.0,27.0,25.0,26.0,97.0,84.0,2018-12-02,2018-12-04,2018-12-01,3
334282,disk_35598,1,18.0,9.0,8.0,9.0,100.0,95.0,2018-12-01,2018-12-02,2018-12-01,1


In [3]:
smartlog_data_fault_A_df['fault_time'] = pd.to_datetime(smartlog_data_fault_A_df['fault_time'])
smartlog_data_fault_A_df['dt'] = pd.to_datetime(smartlog_data_fault_A_df['dt'])
smartlog_data_fault_A_df = smartlog_data_fault_A_df[pd.Timedelta(days=7) >= (smartlog_data_fault_A_df['fault_time']-smartlog_data_fault_A_df['dt'])]

smartlog_data_fault_B_df['fault_time'] = pd.to_datetime(smartlog_data_fault_B_df['fault_time'])
smartlog_data_fault_B_df['dt'] = pd.to_datetime(smartlog_data_fault_B_df['dt'])
smartlog_data_fault_B_df = smartlog_data_fault_B_df[pd.Timedelta(days=7) >= (smartlog_data_fault_B_df['fault_time']-smartlog_data_fault_B_df['dt'])]


In [4]:
print(len(smartlog_data_fault_A_df))

8367


In [5]:
# 将数据存储到CSV文件中去
smartlog_data_fault_A_df.to_csv('smartlog_data_A_train_fault_df.csv', index=False)
smartlog_data_fault_B_df.to_csv('smartlog_data_B_train_fault_df.csv', index=False)